In [1]:
#from IPython.display import display,HTML
from numpy import vectorize as vct
import numpy as np
import optimiser
import grapher
import pricer
import query

In [2]:
qdate = "15/05/2020"
qtime = "12:00"
df = query.opt_chain(qdate,qtime)
df = pricer.impdivsborrows(df,"MidEC","MidEP",0.1)
#display(HTML(df.to_html(index=False)))

American options value (C/P) is composed of European options value (c/p) and early exercise premium (EEP):<br>
$C(S,T) = c(S,T) + EEP_{Call}(S,T)$<br>
$P(S,T) = p(S,T) + EEP_{Put}(S,T)$<br>
<br>
With EEP being derived as an integral of discounted cashflows across time: <br>
$EEP_{Call}(S,T) = \int_{0}^{T} [qB_{t}e^{-q(T-t)}N(d1(S,B_{t},T-t))-rKe^{-r(T-t)}N(d2(S,B_{t},T-t))] dt$<br>
$EEP_{Put}(S,T) = \int_{0}^{T} [rKe^{-r(T-t)}N(-d2(S,B_{t},T-t))-qB_{t}e^{-q(T-t)}N(-d1(S,B_{t},T-t))] dt$<br>
<br>
Spread between American and equivalent European option is the EEP. Cost of carry determines how wide is this spread, in this example underlying will trade ex-div in 45 days, 
so for the front end we shouldn't see a big deviation in value given that underlying is not hard to borrow (as it would push implied yield up). 

In [34]:
df["AmeEurSpreadC"] = df["MidAC"] - df["MidEC"]
df["AmeEurSpreadP"] = df["MidAP"] - df["MidEP"]
grapher.plot2d_quotes(df,"Tenor","Strike","AmeEurSpreadC","AmeEurSpreadP")

Solving european vol which we will use as first guess when solving american vol:<br>
$c(S,T) = Se^{-qT}N(d1(S,K,T))-Ke^{-rT}N(d2(S,K,T))$<br>
$p(S,T) = Ke^{-rT}N(-d2(S,K,T))-Se^{-qT}N(-d1(S,K,T))$

In [35]:
df["MidVolEP"] = vct(pricer.Vol("E","P").root)(df["MidEP"],df["Tenor"],df["MidSpot"],df["Strike"],df["Rate"],df["MeanImpBorrowDiv"],df["Time"])
df["MidVolEC"] = vct(pricer.Vol("E","C").root)(df["MidEC"],df["Tenor"],df["MidSpot"],df["Strike"],df["Rate"],df["MeanImpBorrowDiv"],df["Time"])
df = pricer.interp(df,"MidVolEP","MidVolEC")
grapher.plot2d_quotes(df,"Tenor","Strike","MidVolEC","MidVolEP")

Fitting half-spread function (H) on american quotes which we will later use to adjust american bid along the early exercise boundary (EEB):<br>
$H_{Call}(S) = H_{0} + H_{1}max(S-K_{h},0)$<br>
$H_{Put}(S) = H_{0} + H_{1}max(K_{h}-S,0)$

In [36]:
df = pricer.fit_spread(df,"C","BidAC","AskAC","SpreadParamsAC")
df = pricer.fit_spread(df,"P","BidAP","AskAP","SpreadParamsAP")
df["BidAskSpreadC"] = df["AskAC"] - df["BidAC"]
df["BidAskSpreadP"] = df["AskAP"] - df["BidAP"]
grapher.plot2d_quotes(df,"Tenor","Strike","BidAskSpreadC","BidAskSpreadP")

Terminal condition:<br>
$B_{T,Call} = Kmax(1,\frac{r}{q})$<br>
$B_{T,Put} = Kmin(1,\frac{r}{q})$<br>
<br>
Boundary conditions: high-contact and value-matching:<br>
$\lim_{S\rightarrow B_{t}} \frac{\partial P(S,K,t)}{\partial S} = -1$<br>
$\lim_{S\rightarrow B_{t}} \frac{\partial C(S,K,t)}{\partial S} = 1$<br>
$\lim_{S\rightarrow B_{t}} P(S,K,t) = K - B_{t}$<br>
$\lim_{S\rightarrow B_{t}} C(S,K,t) = B_{t} - K$<br>
<br>
Boundary can be solved successively starting from expiry (using terminal condition) and moving backwards in time:<br>
$B_{t} - K = c(B_{t},T-t) + EEP_{Call}(B_{t},T-t) - H(B_{t})$<br>
$K - B_{t} = p(B_{t},T-t) + EEP_{Put}(B_{t},T-t) - H(B_{t})$<br>
<br>
To determine EEP at each partition (i) we can use trapezoid rule to integrate EEP from i to 0 (expiry):<br>
$\int_{a}^{b}f(x)dx \approx \Delta x[\frac{(f(x_{a}) + f(x_{b}))}{2} + \sum_{n=1}^{N-1}f(x_{n})]$<br>
<br>
And once we have the boundary solve current EEP using simpson rule to integrate from 0 to M:<br>
$\int_{a}^{b}f(x)dx \approx \frac{\Delta x}{3}  [f(x_{a}) + 4f(x_{a+1}) + 2f(x_{a+2}) + ... + 4f(x_{b-1}) + f(x_{b})]$

In [37]:
df["MidVolAC"],df["eepC"],df["eebC"] = vct(pricer.Vol("A","C",8).root)(df["MidAC"],df["Tenor"],df["MidSpot"],df["Strike"],df["Rate"],df["MeanImpBorrowDiv"],df["Time"],df["MidVolEC"],df["SpreadParamsAC"])
df["MidVolAP"],df["eepP"],df["eebP"] = vct(pricer.Vol("A","P",8).root)(df["MidAP"],df["Tenor"],df["MidSpot"],df["Strike"],df["Rate"],df["MeanImpBorrowDiv"],df["Time"],df["MidVolEP"],df["SpreadParamsAP"])
df = pricer.interp(df,"MidVolAC","MidVolAP","eepC","eepP")

In [38]:
grapher.plot3d_boundaries(df,0)

Deep ITM american calls have small extrinsic value left but none have yet reached the boundary at bid, in contrast to european quotes which have more than few strikes below parity:

In [39]:
df["Forward"] = vct(pricer.forward)(df["Tenor"],df["MidSpot"],df["Rate"],df["MeanImpBorrow"],df["CumDivDays"],df["Div"])
grapher.plot3d_quotes(df,"BidSpot","Strike","BidAC",atm_plots=False,div_plot=False,payoff_plot=True,params_plot=False,z_col2="BidEC")

EEP and american-european px call spread; american and european vol:

In [40]:
grapher.plot2d_quotes(df,"Tenor","Strike","eepC","AmeEurSpreadC")
df["MeanVolA"] = (df["MidVolAP"]+df["MidVolAC"])/2
df["MeanVolE"] = (df["MidVolEP"]+df["MidVolEC"])/2
grapher.plot2d_quotes(df,"Tenor","Strike","MeanVolA","MeanVolE")

We can now parametrise this de-americanised vol using SSVI. Where total surface variance is defined as:<br>
$w(k) = \frac{\theta_{t}}{2}(1 + k\rho(\theta_{t})\phi(\theta_{t}) + \sqrt{(k\phi(\theta_{t}) + \rho(\theta_{t}))^2 + (1-\rho(\theta_{t})^2)})$<br><br>
Correlation parameter:<br> 
$\rho(\theta_{t}) = ae^{-b\theta_{t}} + c$<br><br>
Power law: <br>
$\phi(\theta_{t}) = \eta/\theta_{t}^{\gamma}(1 + \theta_{t})^{1-\gamma}$<br><br>
Residual to be minimised:<br>
$\epsilon = (\sigma_{ssvi} - \sigma_{quotes})^2$<br><br>
Where vol is: <br>
$\sigma_{ssvi} = \sqrt{w(k)/t}$

In [41]:
df["Vol"] = df["MeanVolA"] 
df["LogStrike"] = np.log(df["Strike"]/df["Forward"])
df["Moneyness"] = df["Strike"]/df["MidSpot"]
df = pricer.tot_atm_var(df,"LogStrike","Vol","TotAtmVar")
df["TotVar"],df["Var"] = vct(pricer.tot_var)(df["Vol"],df["Tenor"])
df = optimiser.parametrise(df,"MidSpot","Forward","LogStrike","TotAtmVar","Tenor","Vol",2,100,1000)

Vol slice is empty of fly arb when the risk neutral density (RND = d2V/dK2) of that slice is non-negative at any point:<br>
$g(k) = (1-\frac{kw'(k)}{2w(k)})^2-\frac{w'(k)^2}{4}(\frac{1}{w(k)} %2B \frac{1}{4}) %2B \frac{w''(k)}{2}$<br>
$d_{-}(k) = -\frac{k}{\sqrt{w}}\frac{\sqrt{w}}{2}$<br>
$p(k) = \frac{g(k)}{\sqrt{2\pi w(k)}}e^{-\frac{d_{-}(k)}{2}^2}$<br><br>
And condition for absence of calendar arb is for total variances of each slice to not intersect:

In [42]:
grapher.plot2d_fitted(df,"TotAtmVar","Tenor","Forward",2,100,1000)

Intuitive parameters (Jumpwings):<br>
$v_{t} = \frac{\theta_{t}}{t}$<br>
$\psi_{t} = \frac{1}{2}\rho(\theta_{t}) \phi(\theta_{t}) \sqrt{\theta}$<br>
$p_{t} = \frac{1}{2}\sqrt{\theta_{t}}\phi(\theta_{t})(1-\rho)$<br>
$c_{t} = \frac{1}{2}\sqrt{\theta_{t}}\phi(\theta_{t})(1 %2B \rho)$<br>
$\widetilde{v_{t}} = \frac{\theta_{t}}{t}(1-\rho(\theta_{t})^2)$<br>

In [43]:
dfx = df[["Tenor","AtmVar","AtmSkew","PWingSlope","CWingSlope","MinVar","Mean","Variance","Skewness","Kurtosis","StdDev"]].drop_duplicates(["Tenor"])
display(HTML(dfx.to_html(index=False)))

Tenor,AtmVar,AtmSkew,PWingSlope,CWingSlope,MinVar,Mean,Variance,Skewness,Kurtosis,StdDev
35,0.187883,-0.237791,0.667802,0.192219,0.130428,30.720254,19.111095,-0.840343,4.799449,0.459252
63,0.195619,-0.258889,0.703496,0.185717,0.129292,30.027907,33.489966,-0.708827,4.300301,0.453137
126,0.166575,-0.284411,0.744177,0.175354,0.102832,29.983772,55.222538,-0.577442,3.845234,0.411448


In [46]:
grapher.plot3d_quotes(df,"MidSpot","Moneyness","Vol",atm_plots=True,div_plot=True,payoff_plot=False,params_plot=True,z_col2="Var")